In [1]:
from langgraph.graph import StateGraph,START,END
from typing import TypedDict
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv

load_dotenv()
model = ChatGoogleGenerativeAI(model='gemini-1.5-flash-latest')

c:\Users\DELL\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
class LlmState(TypedDict):
    
    topic : str
    ans : str
    outline : str

In [12]:
def OutlineLlm(state: LlmState) -> LlmState:
    topic = state['topic']
    prompt = f'Give me the outline for creating a blog which is feed to the another llm (directly give the outline withoiut any description) for topic: {topic}'
    
    outline = model.invoke(prompt).content
    
    state['outline'] = outline

    return state  

In [13]:
def CreateBlogLlm(state: LlmState) -> LlmState:
    topic = state['topic']
    outline = state['outline']
    prompt = f'Give me the blog for the topic : {topic} and outline : {outline}'
    
    ans = model.invoke(prompt).content
    
    state['ans'] = ans
    
    return state  

In [15]:
# define state
graph = StateGraph(LlmState)

# add nodes
graph.add_node('OutlineLlm',OutlineLlm)
graph.add_node('CreateBlogLlm',CreateBlogLlm)

# add edge
graph.add_edge(START,'OutlineLlm')
graph.add_edge('OutlineLlm','CreateBlogLlm')
graph.add_edge('CreateBlogLlm',END)

# compile
workflow = graph.compile()

# execute graph
output = workflow.invoke({'topic':'Stray Dogs'})

In [16]:
output

{'topic': 'Stray Dogs',
 'ans': "## The Plight of Stray Dogs: A Growing Crisis and the Path to a Solution\n\n**I. Introduction: The Plight of Stray Dogs**\n\nThe heartbreaking sight of a stray dog scavenging for food, shivering in the cold, or injured and alone is a common reality in many parts of the world.  Stray dogs, often overlooked and misunderstood, face a daily struggle for survival.  Their plight is a complex issue with far-reaching consequences, impacting public health, safety, and the environment. This blog post will delve into the causes of stray dog populations, their impact on society, and explore effective solutions to address this growing crisis.\n\n\n**II. Causes of Stray Dog Populations:**\n\nThe proliferation of stray dogs isn't simply a matter of chance; it's a consequence of several interconnected factors:\n\n**A. Abandonment:**  Irresponsible pet ownership is a primary driver.  Dogs are often abandoned due to unwanted litters, financial constraints, moving to unsu

In [18]:
print(output['outline'])

I. Introduction: The Plight of Stray Dogs

II. Causes of Stray Dog Populations:
    A. Abandonment
    B. Uncontrolled Breeding
    C. Lack of Spay/Neuter Programs

III. Impacts of Stray Dogs:
    A. Public Health Concerns (Rabies, etc.)
    B. Traffic Accidents
    C. Environmental Issues

IV. Solutions and Interventions:
    A. Trap-Neuter-Return (TNR) Programs
    B. Adoption and Fostering Initiatives
    C. Public Awareness Campaigns
    D. Responsible Pet Ownership Education
    E. Government Policies and Regulations

V. Success Stories and Case Studies:  (Include links where possible)

VI. Call to Action: How Readers Can Help

VII. Conclusion: A Vision for a More Humane Future for Stray Dogs


In [19]:
print(output['ans'])

## The Plight of Stray Dogs: A Growing Crisis and the Path to a Solution

**I. Introduction: The Plight of Stray Dogs**

The heartbreaking sight of a stray dog scavenging for food, shivering in the cold, or injured and alone is a common reality in many parts of the world.  Stray dogs, often overlooked and misunderstood, face a daily struggle for survival.  Their plight is a complex issue with far-reaching consequences, impacting public health, safety, and the environment. This blog post will delve into the causes of stray dog populations, their impact on society, and explore effective solutions to address this growing crisis.


**II. Causes of Stray Dog Populations:**

The proliferation of stray dogs isn't simply a matter of chance; it's a consequence of several interconnected factors:

**A. Abandonment:**  Irresponsible pet ownership is a primary driver.  Dogs are often abandoned due to unwanted litters, financial constraints, moving to unsuitable housing, or simply a change in lifest